# Experiments related to the personal contribution
EA algorithm to enhance client selection in the federated setting

# Setup
First, import of necessary libraries

In [1]:
import sys

from models.model import LeNet5 
sys.path.append('../data/cifar100/')
from cifar100_loader import CIFAR100DataLoader
from utils.federated_utils import plot_metrics,test, plot_client_selection,save_data,load_data
from EA_algorithm import EA_algorithm

BATCH_SIZE = 64 #constant

# Data loading

In [2]:
#10% of the dataset kept for validation
data_loader = CIFAR100DataLoader(batch_size=BATCH_SIZE, validation_split=0.1, download=True, num_workers=4, pin_memory=True)
trainloader, validloader, testloader = data_loader.train_loader, data_loader.val_loader, data_loader.test_loader

100.0%


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


# Experiment on the iid data distribution

In [3]:
#Best lr and wd found for iid federated baseline: lr=0.1, wd=0.001
global_model = LeNet5()
global_model,val_accuracies,val_losses,train_accuracies,train_losses,client_selection_count=EA_algorithm(1000,10,2,100,0.5,trainloader,validloader,0.1,0.001)
test_accuracy = test(global_model,testloader)
print("Test accuracy: ",test_accuracy)
plot_metrics(train_accuracies,train_losses,val_accuracies,val_losses,"EA_iid")
plot_client_selection(client_selection_count,"EA_iid_client_selection")
save_data(global_model,val_accuracies,val_losses,train_accuracies,train_losses,client_selection_count,"EA_iid")

No checkpoint found, starting from epoch 1.
Generation 2, accuracy 2.1399999999999997, loss 4.512314645549919
Checkpoint saved: ../checkpoints/personal_contribution\model_epoch_10_params_LR0.1_WD0.001.pth


TypeError: Object of type Individual is not JSON serializable